# `nbconvert_html5` interfaces

this project exports tools for making jupyter notebook products more accessible. currently, this project features:
* an `nbconvert` template with that uses html5 semantics.

In [1]:
    from nbconvert import  get_exporter, get_export_names
    import nbconvert_html5

In [2]:
    assert "html5" in get_export_names(), "the entry point is not defined."

In [3]:
    html, html5 = get_exporter("html"), get_exporter("html5")

In [4]:
    A = nbconvert_html5.modifiers.Soup(html().from_filename("notebooks/lorenz.ipynb")[0])
    B = html5().from_filename("notebooks/lorenz-executed.ipynb")[0]

there is a `main` landmark

In [5]:
    A.select("section"), len(B.select("main"))

([], 1)

cells are articles

In [6]:
    A.select("article"), len(B.select("article.jp-Cell"))

([], 16)

outputs are sections

In [7]:
    A.select("section"), len(B.select("section.jp-OutputArea"))

([], 5)

In [9]:
__import__("pathlib").Path("better.html").write_text(str(B));